In [1]:
!sudo apt-get -y install libopenmpi-dev && pip3 install --upgrade pip setuptools && pip3 install tensorrt_llm --extra-index-url https://pypi.nvidia.com

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
The following package was automatically installed and is no longer required:
  python3-distro-info
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Cloning https://github.com/flashinfer-ai/flashinfer.git (to revision 06309c4e) to /tmp/pip-install-lkoylxxz/flashinfer_ca2212c3812c48289af970cd2c223eb0
  Running command git clone --filter=blob:none --quiet https://github.com/flashinfer-ai/flashinfer.git /tmp/pip-install-lkoylxxz/flashinfer_ca2212c3812c48289af970cd2c223eb0
  Running command git checkout -q 06309c4e
  Resolved https://github.com/flashinfer-a

In [2]:
!git clone https://github.com/NVIDIA/TensorRT-LLM.git

fatal: destination path 'TensorRT-LLM' already exists and is not an empty directory.


In [5]:
cd TensorRT-LLM/tensorrt_llm/_torch/models/

/home/ubuntu/TensorRT-LLM/tensorrt_llm/_torch/models


tensorrt_llm/_torch/models/modeling_llama.py: ~200 LoC

LlamaAttention: ~30 LoC

LlamaDecoderLayer: ~60 LoC

LlamaModel: ~50 LoC

LlamaForCausalLM: ~15 LoC

In [6]:
from typing import Optional, Tuple

import torch
from torch import nn
from transformers import LlamaConfig
from tensorrt_llm.functional import PositionEmbeddingType

from tensorrt_llm._torch.attention_backend import AttentionMetadata
from tensorrt_llm._torch.attention_backend.interface import PositionalEmbeddingParams, RopeParams
from tensorrt_llm._torch.distributed import ParallelConfig, TensorParallelMode
from tensorrt_llm._torch.model_config import ModelConfig
from tensorrt_llm._torch.modules.attention import Attention
from tensorrt_llm._torch.modules.decoder_layer import DecoderLayer
from tensorrt_llm._torch.modules.embedding import Embedding
from tensorrt_llm._torch.modules.gated_mlp import GatedMLP
from tensorrt_llm._torch.modules.rms_norm import RMSNorm
from tensorrt_llm._torch.modules.rotary_embedding import RotaryEmbedding
from tensorrt_llm._torch.models.modeling_utils import (DecoderModel, DecoderModelForCausalLM,
                             register_auto_model)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 22:09:21.299926: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 22:09:21.487555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742940561.567752   93479 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742940561.591223   93479 cuda_blas.cc:1418] Unable to 

[TensorRT-LLM] TensorRT-LLM version: 0.17.0.post1


/home/ubuntu/.local/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Note that MyAttention inherits from our Attention module (in tensorrt_llm/_torch/modules/attention.py), so that the attention computation is compatible with our PyTorch runtime. Related to this, module inputs should also be adapted:

The attn_metadata stores the metadata from the batched input and KV cache for the attention backend. It is created by and passed from the runtime, and model developers need to ensure that attn_metadata is correctly passed to the attention module.

The input tensors (i.e., input_ids, position_ids, hidden_states) are in the packed mode. The first dimension corresponds to the number of tokens in a batch.

In [7]:
!python -c "import torch; print(torch.__version__)"
!pip show tensorrt-llm

2.5.1+cu124
Name: tensorrt-llm
Version: 0.17.0.post1
Summary: TensorRT-LLM: A TensorRT Toolbox for Large Language Models
Home-page: https://github.com/NVIDIA/TensorRT-LLM
Author: NVIDIA Corporation
Author-email: 
License: Apache License 2.0
Location: /home/ubuntu/.local/lib/python3.10/site-packages
Requires: accelerate, aenum, build, click, click-option-group, colored, cuda-python, diffusers, evaluate, fastapi, flashinfer, h5py, httpx, lark, mpi4py, mpmath, numpy, nvidia-cuda-nvrtc-cu12, nvidia-modelopt, nvidia-nccl-cu12, onnx, onnx-graphsurgeon, openai, optimum, ordered-set, pandas, pillow, polygraphy, psutil, pulp, pydantic, pynvml, pyzmq, s2wrapper, sentencepiece, setuptools, StrEnum, tensorrt, torch, torchvision, transformers, uvicorn, wheel
Required-by: 


In [8]:
class LlamaAttention(Attention):
    def __init__(
        self,
        model_config: ModelConfig[LlamaConfig],
        layer_idx: Optional[int] = None,
    ):
        config = model_config.pretrained_config
        if model_config.fuse_pos_embd:
            pos_embd_params = PositionalEmbeddingParams(
                type=PositionEmbeddingType.rope_gpt_neox,
                rope=RopeParams.from_config(config),
            )
        else:
            pos_embd_params = None
        super().__init__(
            hidden_size=config.hidden_size,
            num_attention_heads=config.num_attention_heads,
            num_key_value_heads=config.num_key_value_heads,
            max_position_embeddings=config.max_position_embeddings,
            bias=config.attention_bias,
            rotary_emb=LlamaRotaryEmbedding(config),
            pos_embd_params=pos_embd_params,
            layer_idx=layer_idx,
            dtype=config.torch_dtype,
            config=model_config,
        )

Additionally, MyDecoderLayer, MyModel, and MyModelForCausalLM are subclasses of DecoderLayer, DecoderModel, and DecoderModelForCausalLM respectively. The base classes define interfaces and provide a generic scaffolding to define model layers, load weights, etc.

In [9]:
class LlamaDecoderLayer(DecoderLayer):

    def __init__(
        self,
        model_config: ModelConfig[LlamaConfig],
        layer_idx: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        super().__init__()
        config = model_config.pretrained_config
        self.self_attn = LlamaAttention(
            model_config,
            layer_idx=layer_idx,
        )

        self.mlp = GatedMLP(
            hidden_size=config.hidden_size,
            intermediate_size=config.intermediate_size,
            bias=config.mlp_bias,
            dtype=config.torch_dtype,
            config=model_config,
        )
        self.input_layernorm = RMSNorm(hidden_size=config.hidden_size,
                                       eps=config.rms_norm_eps,
                                       dtype=config.torch_dtype)
        self.post_attention_layernorm = RMSNorm(hidden_size=config.hidden_size,
                                                eps=config.rms_norm_eps,
                                                dtype=config.torch_dtype)

    def forward(
        self,
        position_ids: torch.LongTensor,
        hidden_states: torch.Tensor,
        attn_metadata: AttentionMetadata,
        residual: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> torch.Tensor:
        if residual is None:
            residual = hidden_states
            hidden_states = self.input_layernorm(hidden_states)
        else:
            hidden_states, residual = self.input_layernorm(
                hidden_states, residual)

        # Self Attention
        hidden_states = self.self_attn(
            position_ids=position_ids,
            hidden_states=hidden_states,
            attn_metadata=attn_metadata,
            **kwargs,
        )

        # Fully Connected
        hidden_states, residual = self.post_attention_layernorm(
            hidden_states, residual)
        hidden_states = self.mlp(hidden_states)
        return hidden_states, residual

In [10]:
class LlamaModel(DecoderModel):

    def __init__(self, model_config: ModelConfig[LlamaConfig]):
        super().__init__(model_config)
        config = self.model_config.pretrained_config
        self.padding_idx = config.pad_token_id

        self.embed_tokens = Embedding(
            config.vocab_size,
            config.hidden_size,
            dtype=config.torch_dtype,
            parallel_config=ParallelConfig(
                tensor_parallel_rank=model_config.mapping.tp_rank,
                tensor_parallel_size=model_config.mapping.tp_size,
                tensor_parallel_mode=TensorParallelMode.COLUMN,
                gather_output=True,
                gpus_per_node=model_config.mapping.gpus_per_node,
            ),
        )
        self.layers = nn.ModuleList([
            LlamaDecoderLayer(
                model_config,
                layer_idx,
            ) for layer_idx in range(config.num_hidden_layers)
        ])
        self.norm = RMSNorm(hidden_size=config.hidden_size,
                            eps=config.rms_norm_eps,
                            dtype=config.torch_dtype)

    def forward(
        self,
        attn_metadata: AttentionMetadata,
        input_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
    ) -> torch.Tensor:
        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError(
                "You cannot specify both input_ids and inputs_embeds at the same time, and must specify either one"
            )

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids)

        hidden_states = inputs_embeds

        residual = None
        for decoder_layer in self.layers:
            hidden_states, residual = decoder_layer(position_ids=position_ids,
                                                    hidden_states=hidden_states,
                                                    attn_metadata=attn_metadata,
                                                    residual=residual)

        hidden_states, _ = self.norm(hidden_states, residual)
        return hidden_states

Optionally, you may replace the native PyTorch modules with our implementations to enable features or achieve higher performance:

Linear (in tensorrt_llm/_torch/modules/linear.py): Enables tensor parallelism and quantization.

Embedding (in tensorrt_llm/_torch/modules/embedding.py): Enables tensor parallelism for embedding.

RotaryEmbedding (in tensorrt_llm/_torch/modules/rotary_embedding.py): Enables performant rotary embedding.

RMSNorm (in tensorrt_llm/_torch/modules/rms_norm.py): Enables performant RMS norm.

In [11]:
class LlamaRotaryEmbedding(RotaryEmbedding):

    def __init__(
        self,
        config: LlamaConfig,
        device: Optional[torch.device] = None,
    ):
        super().__init__(
            config,
            head_dim=config.hidden_size // config.num_attention_heads,
            num_attention_heads=config.num_attention_heads,
            max_position_embeddings=config.max_position_embeddings,
            device=device,
            rope_type="default" if config.rope_scaling is None else "llama3")

Model Registration

The new model needs to be registered so that it can be recognized by the PyTorch runtime. The registration can be done simply by adding the register_auto_model decorator for MyModelForCausalLM:

In [12]:
@register_auto_model("LlamaForCausalLM")
class LlamaForCausalLM(DecoderModelForCausalLM[LlamaModel, LlamaConfig]):

    def __init__(
        self,
        model_config: ModelConfig[LlamaConfig],
    ):
        super().__init__(LlamaModel(model_config),
                         config=model_config,
                         hidden_size=model_config.pretrained_config.hidden_size,
                         vocab_size=model_config.pretrained_config.vocab_size)


@register_auto_model("MistralForCausalLM")
class MistralForCausalLM(DecoderModelForCausalLM[LlamaModel, LlamaConfig]):

    def __init__(
        self,
        model_config: ModelConfig[LlamaConfig],
    ):
        super().__init__(LlamaModel(model_config),
                         config=model_config,
                         hidden_size=model_config.pretrained_config.hidden_size,
                         vocab_size=model_config.pretrained_config.vocab_size)